In [10]:
from mongo_cli import db,MODE,db_dev
sid_info = db["sid_info"]
res = sid_info.find({'status':0}).sort("_id",-1).limit(3)
sid_list = [x.get('sid') for x in res]
print(sid_list)

['SID9e07f43176a044ca9b2e9caa58b40ec2', 'SID17a5ddd38e8ed52c2da8ce05220ba87b', 'SID77aefb47bb16acad0fdb24d95dd008a0']


In [31]:
import pandas as pd
import uuid
import json
def df_to_dict(df):
        """不保留index"""
        value = df.values.tolist()
        title = df.columns.tolist()
        return [ dict(zip(title,i)) for i in value]

def out_rpt_sid_info():
    sid_info = db["sid_info"]
    report = db["report"]
    #cmcc = sid_info.find({'status': 0, 'tel_info.flow_type': '10086', 'status_report':0},{'sid':1, '_id':0}).sort("_id", -1).limit(10)
    #cucc = sid_info.find({'status': 0, 'tel_info.flow_type': '10010', 'status_report':0},{'sid':1, '_id':0}).sort("_id", -1).limit(10)
    #ctcc = sid_info.find({'status': 0, 'tel_info.flow_type': '10000', 'status_report':0},{'sid':1, '_id':0}).sort("_id", -1).limit(10)
    #sid_list = [x.get('sid') for x in list(cmcc)+list(cucc)+list(ctcc)]
    sid_list = []
    #for i in ['移动', '联通', '电信']:
    for i in ['电信']:
        for j,k in enumerate([1,1,1,1]):
            tmp = db['report'].find({'cuishou_risk_detection.detection_result':j, 
#                                      'call_behavior.number_used.telecom': i, 
                                     'call_log_group_by_tel':{'$exists':True},
                                     'user_info.conclusion_of_3_elements_check':{'$exists':False},
                                     '$where':'this.call_log_group_by_tel.length<350'}).sort("_id", -1).limit(k)
            sid_list += list(tmp)
#         print(sid_list)
    sid_list = [x.get('sid') for x in sid_list]
    #sid_list = ["SID2bb40e2382f34cfc92f5d8324ac063d8", "SIDdd621b2b90de4e7591961c46e8b42798", "SIDcc8e21db1f1147788af63845a427e8a8", "SIDe50dc7a46a8f4f32ad010ac03a1e9dad", "SID93ab6d123a394e0cadc39f790d5622aa"]
    print(sid_list)
    pat_tel = lambda x: x[:-4]+'*'*len(x[-4:]) if len(x)>5 else '*'*len(x)
    pat_mix = lambda x: x[:1] + '*'*len(x[1:]) if (x and '运营商' not in x) else '***'
    pat_open_date = lambda x: str(int(float(x))-86400*30*3) if (x and '运营商' not in x) else '运营商未透露'
    for i in sid_list:
        print('正在插入：{}'.format(i))
        sid_info = db["sid_info"].find_one({'sid':i},{'_id':0, 'job_id':0})
        phone = db["phone"].find_one({'sid':i},{'_id':0})
        report = db['report'].find_one({'sid':i},{'_id':0})
        # 处理sid_info
        sid_info['cid'] = '1'
        sid = 'SID{}'.format(uuid.uuid4()).replace('-', '') 
        sid_info['sid'] = sid
        print('new_sid:{}'.format(sid))
        # 处理phone
        phone['sid'] = sid
        phone['tel'] = pat_tel(phone.get('tel'))
        info = phone.get('info')
        open_date = pat_open_date(str(info.get('open_date')))
        phone['info']['full_name'] = pat_mix(info.get('full_name'))
        phone['info']['id_card'] = pat_mix(info.get('id_card'))
        phone['info']['open_date'] = open_date
        phone['info']['address'] = pat_mix(info.get('address'))
        calls = pd.DataFrame(phone.get('calls'))
        calls['tel'] = calls['tel'].map(pat_tel)
        calls['call_tel'] = calls['call_tel'].map(pat_tel)
        phone['calls'] = df_to_dict(calls)
        smses = pd.DataFrame(phone.get('smses'))
        smses['peer_number'] = smses['peer_number'].map(pat_tel) if not smses.empty else []
        phone['smses'] = df_to_dict(smses)        
        # 处理report
        tel_info = report.get('tel_info')
        if tel_info:
            report['tel_info']['full_name'] = pat_mix(tel_info.get('full_name'))
            report['tel_info']['id_card'] = pat_mix(tel_info.get('id_card'))
            report['tel_info']['open_date'] = open_date
            report['tel_info']['innet_months'] = str(int(tel_info.get('innet_months'))+3) if ('运营商' not in tel_info.get('innet_months')) else '运营商未透露'
            report['tel_info']['address'] = pat_mix(tel_info.get('address'))
            report['tel_info']['tel'] = pat_tel(tel_info.get('tel'))
        user_info = report.get('user_info')
        if  user_info:
            report['user_info']['user_idcard'] = pat_mix(user_info.get('user_idcard'))
            report['user_info']['user_address'] = pat_mix(user_info.get('user_address'))
            report['user_info']['user_name'] = pat_mix(user_info.get('user_name'))
        if user_info.get('user_tel'):
            report['user_info']['user_tel'] = pat_mix(user_info.get('user_tel'))
        call_by_tel = report.get('call_log_group_by_tel')
        if call_by_tel:
            [ i.update({'format_tel':pat_tel(i.get('format_tel'))}) for i in call_by_tel]
            report['call_log_group_by_tel'] = call_by_tel
        contact = report.get('calls_sa_by_major_contact')
        if contact:
            [ i.update({'format_tel':pat_tel(i.get('format_tel'))}) for i in contact]
            report['calls_sa_by_major_contact'] = contact
        call_by_length = report.get('calls_sa_by_length')
        if call_by_length:
            [ i.update({'format_tel':pat_tel(i.get('format_tel'))}) for i in call_by_length]
            report['calls_sa_by_length'] = call_by_length
        call_by_times = report.get('calls_sa_by_times')
        if call_by_times:
            [ i.update({'format_tel':pat_tel(i.get('format_tel'))}) for i in call_by_times]
            report['calls_sa_by_times'] = call_by_times
        mergency_contact = report.get('mergency_contact')
        if mergency_contact:
            [ i.update({'format_tel':pat_tel(i.get('format_tel')),'contact_name':pat_mix(i.get('contact_name'))}) for i in mergency_contact]
            report['mergency_contact'] = mergency_contact
        call_behavior = report.get('call_behavior')
        if call_behavior:
            report['call_behavior']['number_used']['open_date'] = pat_open_date(str(call_behavior.get('number_used').get('open_date')))
            number_tags = call_behavior.get('number_tags')
            if number_tags:
                [ i.update({'tel':pat_tel(i.get('tel'))}) for i in number_tags]
                report['call_behavior']['number_tags'] = number_tags
        report['sid']=sid
        db_dev['report'].delete_many({'sid':sid})
        db_dev['call_log'].delete_many({'sid':sid})
        db_dev['sid_info'].delete_many({'sid':sid})
        db_dev['sid_info'].update({'sid':sid},sid_info, upsert=True)
        
        with open('files/{}_phone.json'.format(sid), 'w') as fp:
            json.dump(phone, fp, indent=4, sort_keys=True)
        with open('files/{}_rpt.json'.format(sid), 'w') as fp:
            json.dump(report, fp, indent=4, sort_keys=True)
        db_dev['phone'].update({'sid':sid}, phone, upsert=True)
        db_dev['report'].update({'sid':sid},report, upsert=True)
        print("完成插入")

out_rpt_sid_info()

['SIDffcef201be64453f82beedfa86067113', 'SIDc3d62a1a217c49cb9bf9bdd587d6726e', 'SID1b888921cf6f4d6468509e2e8d9706b1', 'SID46fc8ba682af430bb9ad2f3d778fb3e3']
正在插入：SIDffcef201be64453f82beedfa86067113
new_sid:SID62732998a51945b79f3649dafbfb14a9


C:\Users\Yulore\AppData\Local\conda\conda\envs\py35\lib\site-packages\ipykernel_launcher.py:107: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
C:\Users\Yulore\AppData\Local\conda\conda\envs\py35\lib\site-packages\ipykernel_launcher.py:113: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
C:\Users\Yulore\AppData\Local\conda\conda\envs\py35\lib\site-packages\ipykernel_launcher.py:114: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.


完成插入
正在插入：SIDc3d62a1a217c49cb9bf9bdd587d6726e
new_sid:SIDd0580442255d4f2ba23da09b3842795a
完成插入
正在插入：SID1b888921cf6f4d6468509e2e8d9706b1
new_sid:SIDcdd940c508344280bca04a6e8207627e
完成插入
正在插入：SID46fc8ba682af430bb9ad2f3d778fb3e3
new_sid:SIDfe07dd0a9f4e4b158fa19d6d53145f89
完成插入
